In [1]:
import cv2
import pickle
import os.path
import matplotlib.pyplot as plt
from imutils import paths
from sklearn.preprocessing import LabelBinarizer
import numpy as np
from sklearn.model_selection import train_test_split
from keras.layers import Dense,GlobalAveragePooling2D,Dropout
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.models import model_from_json
from sklearn import metrics
from sklearn.metrics import roc_curve,auc,accuracy_score,classification_report
from keras.optimizers import SGD
from keras import models
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
import glob

Using TensorFlow backend.


In [2]:
!git clone https://github.com/magus96/histology_classifier.git

Cloning into 'histology_classifier'...
remote: Enumerating objects: 104, done.
remote: Counting objects: 100% (104/104), done.
remote: Compressing objects: 100% (101/101), done.
remote: Total 104 (delta 4), reused 90 (delta 0), pack-reused 0
Receiving objects: 100% (104/104), 4.18 MiB | 29.29 MiB/s, done.
Resolving deltas: 100% (4/4), done.


In [3]:
cd '/content/histology_classifier/'

/content/histology_classifier


In [0]:
path = glob.glob("/content/histology_classifier/data_chec_resized/Skin/*.jpg")

for img in path:
    n = cv2.imread(img)
    cv_img.append(n)
    label=img.split(os.path.sep)[-2]
    labels.append(label)

In [12]:
data = np.array(cv_img, dtype="float")/ 255.0
labels=np.array(labels)
print(data.shape)
print(labels.shape)
print(labels)

(79, 192, 256, 3)
(79,)
['Bladder' 'Bladder' 'Bladder' 'Bladder' 'Bladder' 'Bladder' 'Bladder'
 'Bladder' 'Bladder' 'Bladder' 'Bladder' 'Brain' 'Brain' 'Brain' 'Brain'
 'Brain' 'Brain' 'Brain' 'Brain' 'Brain' 'Brain' 'Kidney' 'Kidney'
 'Kidney' 'Kidney' 'Kidney' 'Kidney' 'Kidney' 'Kidney' 'Kidney' 'Kidney'
 'LI' 'LI' 'LI' 'LI' 'LI' 'LI' 'LI' 'LI' 'LI' 'Liver' 'Liver' 'Liver'
 'Liver' 'Liver' 'Liver' 'Liver' 'Liver' 'Liver' 'Liver' 'Lung' 'Lung'
 'Lung' 'Lung' 'Lung' 'Lung' 'Lung' 'Lung' 'Lung' 'Lung' 'Lung' 'SI' 'SI'
 'SI' 'SI' 'SI' 'SI' 'SI' 'SI' 'SI' 'SI' 'SI' 'Skin' 'Skin' 'Skin' 'Skin'
 'Skin' 'Skin' 'Skin']


In [13]:
lb = LabelBinarizer().fit(labels)
labels = lb.transform(labels)

print(labels.shape)

(79, 8)


In [0]:
X_train,X_test,y_train,y_test=train_test_split(data,labels,test_size=0.2,random_state=42)

In [15]:

print(X_train.shape)

(63, 192, 256, 3)


In [16]:
vgg_conv = VGG16(weights='imagenet',include_top=False,input_shape=(192, 256, 3))

Instructions for updating:
Colocations handled automatically by placer.
58892288/58889256 [==============================] - 1s 0us/step


In [0]:
nTrain=63

In [0]:
train_features = np.zeros(shape=(nTrain, 6,8,512))
#datagen.fit(data)


In [19]:
train_features=vgg_conv.predict(X_train)
print(train_features.shape)

(63, 6, 8, 512)


In [0]:

train_features = np.reshape(train_features, (nTrain, 6 * 8 * 512))

In [0]:
from sklearn.linear_model import LogisticRegression


In [22]:
!pip install scikit-multilearn

    100% |████████████████████████████████| 92kB 3.5MB/s 


In [0]:
from skmultilearn.problem_transform import ClassifierChain
classifier = ClassifierChain(LogisticRegression())

In [24]:
classifier.fit(train_features,y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  Futur

ClassifierChain(classifier=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False),
        order=None, require_dense=[True, True])

In [0]:
test_features=np.zeros(shape=(16,6,8,512))

In [26]:
test_features=vgg_conv.predict(X_test)
print(test_features.shape)

(16, 6, 8, 512)


In [0]:
test_features = np.reshape(test_features, (16, 6 * 8 * 512))

In [28]:
print(y_test)

[[0 0 1 0 0 0 0 0]
 [1 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0]
 [0 0 0 1 0 0 0 0]
 [0 1 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0]
 [1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0]
 [1 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0]
 [0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 1 0]
 [0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 1 0]
 [0 0 0 0 1 0 0 0]]


In [29]:
pred2=classifier.predict(test_features)
print(pred2)

  (6, 0)	1.0
  (8, 0)	1.0
  (4, 1)	1.0
  (9, 1)	1.0
  (10, 4)	1.0
  (15, 4)	1.0
  (12, 6)	1.0


In [30]:
print("Accuracy = ",accuracy_score(y_test,pred2))
print("\n")

Accuracy =  0.4375


